In [1]:
import os
import shutil
import open3d
import numpy as np
import tqdm
import glob
import copy

import utils.registration as registration
import utils.pointcloud as pointcloud
import utils.fread as fread
import utils.functions as functions

from utils.depth_camera import DepthCamera

In [4]:
depth_camera = DepthCamera(metadata="../local-registration/data/raw_data/exp_11/metadata/device-3-aligned.json", name="device-3-aligned")

In [4]:
depth_img_file = "../local-registration/data/raw_data/exp_11/trial_1/secondary/subject-1/02/frames/frame-1680509281682.depth.png"

pcd = depth_camera.depth_to_point_cloud(depth_img_file)

In [32]:
pcd = copy.deepcopy(pcd)

pcd = open3d.geometry.voxel_down_sample(pcd, 0.03)

# x = open3d.geometry.uniform_down_sample(x, every_k_points=5)

_, ind = open3d.geometry.statistical_outlier_removal(pcd, nb_neighbors=80, std_ratio=0.5)
pcd = open3d.geometry.select_down_sample(pcd, ind)

In [33]:
pointcloud.view(pcd)

In [11]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = open3d.geometry.select_down_sample(cloud, ind)
    outlier_cloud = open3d.geometry.select_down_sample(cloud, ind, invert=True)

    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    
    open3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [12]:
def remove_outliers(pcd, nb_neighbors=80, std_ratio=0.5):
    _, ind = open3d.geometry.statistical_outlier_removal(pcd, nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    return open3d.geometry.select_down_sample(pcd, ind)

In [14]:
dataset_dir = "../local-registration/data/raw_data/exp_10/trial_1/secondary/subject-1/02/frames"

files = np.random.choice(glob.glob(os.path.join(dataset_dir, "*.depth.png")), 10, replace=False)

In [13]:
for file in files:
    pcd = depth_camera.depth_to_point_cloud(file)
    
    x = copy.deepcopy(pcd)
    x = open3d.geometry.voxel_down_sample(x, 0.03)
    x = remove_outliers(x)
    pointcloud.view(x)
    
    # display_inlier_outlier(x, ind)

In [15]:
for file in files:
    pcd = depth_camera.depth_to_point_cloud(file)
    pointcloud.view(pcd)